In [2]:
import torch
from tqdm.notebook import tqdm
!pip install diffusers
 #install diffusers==0.23.1 #--upgrade diffusers[torch]
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline

device = "cuda"

In [8]:
from google.colab import files

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install pyarrow==11.0.0
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 30.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 15.0.0
    Uninstalling pyarrow-15.0.0:
      Successfully uninstalled pyarrow-15.0.0

In [ ]:
images_dataset = load_dataset("premai-io/sd-ml-assignment")
image_caption = dict()

for image_nr in range(0, 90):
    image_caption[image_nr] = {'prompt' :images_dataset["data"][image_nr]['prompt'].replace('Charming low-res pixel art inspired image of ', 'mfs01_').replace(' using pastel pink, pastel red, pastel purple, and yellow shadings', '')}
    images_dataset["data"][image_nr]['image'].save(f"./drive/MyDrive/Colab Notebooks/SDXL/Train Images/{image_caption[image_nr]}.png")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating data split: 0 examples [00:00, ? examples/s]

In [ ]:
!pip install -U autotrain-advanced


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.5/226.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 77.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 75.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━

In [ ]:
!pip install accelerate
from accelerate import Accelerator

In [3]:
import os

# project configuration
project_name = "SDXL"

model_name_base = "stabilityai/stable-diffusion-xl-base-1.0"
model_name_refiner = "stabilityai/stable-diffusion-xl-refiner-1.0"

# fine-tuning prompts
instance_prompt = 'Charming low-res pixel art inspired image using pastel pink, pastel red, pastel purple, and yellow shadings ' #"/content/drive/MyDrive/Colab Notebooks/SDXL/Prompts/prompts.json"
class_prompt = "mfs01 artstyle"
image_path = "/content/drive/MyDrive/Colab Notebooks/SDXL/Train Images/"

# fine-tuning hyperparameters
learning_rate = 4e-4
num_steps = 500
batch_size = 1
gradient_accumulation = 4
resolution = 1024

# environment variables for autotrain command
os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = model_name_base
os.environ["INSTANCE_PROMPT"] = instance_prompt
os.environ["CLASS_PROMPT"] = class_prompt
os.environ["IMAGE_PATH"] = image_path
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_STEPS"] = str(num_steps)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["RESOLUTION"] = str(resolution)

In [ ]:
pipeline = DiffusionPipeline.from_pretrained(
    model_name_base,
    torch_dtype=torch.float16,
).to(device)

refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    model_name_refiner,
    torch_dtype=torch.float16,
).to(device)

subject_prompt = """Charming art inspired image of cat using pastel pink, pastel red, pastel purple, and yellow shadings"""
refiner_prompt = """low-res pixel"""


for seed in range(0, 1):
    generator = torch.Generator(device).manual_seed(seed)
    base_image = pipeline(
        prompt=f"{subject_prompt}, {refiner_prompt}",
        negative_prompt=f"{subject_negative_prompt}, {refiner_negative_prompt}",
        num_inference_steps=20,
        generator=generator,
        height=1024,
        width=1024,
        output_type="image",
    ).images[0]

    refined_image = refiner(
        prompt=refiner_prompt,
        negative_prompt=refiner_negative_prompt,
        num_inference_steps=4,
        generator=generator,
        image=base_image,
    ).images[0]

    refined_image.save(f"cat_base_model_{seed}.png")
    files.download(f"cat_base_model_{seed}.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
!autotrain dreambooth --help

usage: autotrain <command> [<args>] dreambooth [-h] [--revision REVISION] [--tokenizer TOKENIZER]
                                               --image-path IMAGE_PATH
                                               [--class-image-path CLASS_IMAGE_PATH] --prompt
                                               PROMPT [--class-prompt CLASS_PROMPT]
                                               [--num-class-images NUM_CLASS_IMAGES]
                                               [--class-labels-conditioning CLASS_LABELS_CONDITIONING]
                                               [--prior-preservation]
                                               [--prior-loss-weight PRIOR_LOSS_WEIGHT]
                                               --resolution RESOLUTION [--center-crop]
                                               [--train-text-encoder]
                                               [--sample-batch-size SAMPLE_BATCH_SIZE]
                                               [--num-steps NUM_

In [ ]:
!autotrain dreambooth \
    --model ${MODEL_NAME} \
    --project-name ${PROJECT_NAME} \
    --image-path "${IMAGE_PATH}" \
    --prompt "${INSTANCE_PROMPT}" \
    --resolution ${RESOLUTION} \
    --batch-size ${BATCH_SIZE} \
    --num-steps ${NUM_STEPS} \
    --gradient-accumulation ${GRADIENT_ACCUMULATION} \
    --lr ${LEARNING_RATE} \
    --fp16

> INFO    Namespace(version=False, revision=None, tokenizer=None, image_path='/content/drive/MyDrive/Colab Notebooks/SDXL/Train Images/', class_image_path=None, prompt='Charming low-res pixel art inspired image using pastel pink, pastel red, pastel purple, and yellow shadings ', class_prompt=None, num_class_images=100, class_labels_conditioning=None, prior_preservation=None, prior_loss_weight=1.0, resolution=1024, center_crop=None, train_text_encoder=None, sample_batch_size=4, num_steps=500, checkpointing_steps=100000, resume_from_checkpoint=None, scale_lr=None, scheduler='constant', warmup_steps=0, num_cycles=1, lr_power=1.0, dataloader_num_workers=0, use_8bit_adam=None, adam_beta1=0.9, adam_beta2=0.999, adam_weight_decay=0.01, adam_epsilon=1e-08, max_grad_norm=1.0, allow_tf32=None, prior_generation_precision=None, local_rank=-1, xformers=None, pre_compute_text_embeddings=None, tokenizer_max_length=None, text_encoder_use_attention_mask=None, rank=4, xl=None, fp16=True, bf16=None, vali

In [5]:
!pip install transformers==4.28.0
!pip install accelerate==0.17.0
import transformers
import accelerate
from accelerate import Accelerator

pipeline = DiffusionPipeline.from_pretrained(
    model_name_base,
    torch_dtype=torch.float16,
).to(device)

pipeline.load_lora_weights(
    project_name,
    weight_name="pytorch_lora_weights.safetensors"
)

refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    model_name_refiner,
    torch_dtype=torch.float16,
).to(device)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 2.1 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

text_encoder/config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/9.04G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [14]:
subject_prompt = """Charming low-res pixel art inspired mfs01 image of pickle using pastel pink, pastel red, pastel purple, and yellow shadings"""

subject_negative_prompt = """high resolution, background"""

refiner_prompt = """low res"""

refiner_negative_prompt = """high quality, high-resolution"""

for seed in range(0, 3):
    generator = torch.Generator(device).manual_seed(seed)
    base_image = pipeline(
        prompt=f"{subject_prompt}, {refiner_prompt}",
        negative_prompt=f"{subject_negative_prompt}, {refiner_negative_prompt}",
        num_inference_steps=100,
        generator=generator,
        height=1024,
        width=1024,
        output_type="latent",
    ).images[0]

    refined_image = refiner(
        prompt=refiner_prompt,
        negative_prompt=refiner_negative_prompt,
        num_inference_steps=20,
        generator=generator,
        image=base_image,
    ).images[0]

    refined_image.save(f"pickle_trained_{seed}.png")
    files.download(f"pickle_trained_{seed}.png")

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  0%|          | 0/100 [00:00<?, ?it/s]

...

<IPython.core.display.Javascript object>

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>